<a href="https://colab.research.google.com/github/mdxsuhail/JEE-kcet-comdk-rank-predictor/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings

# --- 1. CONFIGURATION ---
warnings.filterwarnings("ignore", category=UserWarning) # Silence sklearn warnings

# --- 2. THE MASTER DATABASE (Consolidated from all phases) ---
# Includes: Top Tier (RVCE/BMS), Tier 2 (Sir MVIT/CMRIT), Tier 3, and Universities.
# Source: Official Round 3 2025 Cutoffs (COMEDK)

master_data = [
    # --- TIER 1: THE GIANTS ---
    { "College": "RV College of Engineering", "Branch": "Computer Science (CSE)", "COMEDK_Rank": 542, "KCET_Rank": 250 },
    { "College": "RV College of Engineering", "Branch": "Data Science", "COMEDK_Rank": 649, "KCET_Rank": 350 },
    { "College": "RV College of Engineering", "Branch": "AI & Machine Learning", "COMEDK_Rank": 713, "KCET_Rank": 400 },
    { "College": "RV College of Engineering", "Branch": "Cyber Security", "COMEDK_Rank": 774, "KCET_Rank": 450 },
    { "College": "RV College of Engineering", "Branch": "Electronics (ECE)", "COMEDK_Rank": 1077, "KCET_Rank": 900 },
    { "College": "RV College of Engineering", "Branch": "Electrical (EEE)", "COMEDK_Rank": 1866, "KCET_Rank": 2000 },
    { "College": "RV College of Engineering", "Branch": "Aerospace Engineering", "COMEDK_Rank": 3464, "KCET_Rank": 4000 },
    { "College": "RV College of Engineering", "Branch": "Mechanical", "COMEDK_Rank": 5339, "KCET_Rank": 5500 },

    { "College": "MS Ramaiah (MSRIT)", "Branch": "Computer Science (CSE)", "COMEDK_Rank": 1341, "KCET_Rank": 900 },
    { "College": "MS Ramaiah (MSRIT)", "Branch": "AI & Machine Learning", "COMEDK_Rank": 1649, "KCET_Rank": 1200 },
    { "College": "MS Ramaiah (MSRIT)", "Branch": "Information Science (ISE)", "COMEDK_Rank": 2311, "KCET_Rank": 1500 },
    { "College": "MS Ramaiah (MSRIT)", "Branch": "Electronics (ECE)", "COMEDK_Rank": 3162, "KCET_Rank": 2500 },

    { "College": "BMS College of Engineering", "Branch": "Computer Science (CSE)", "COMEDK_Rank": 3689, "KCET_Rank": 1000 },
    { "College": "BMS College of Engineering", "Branch": "AI & Data Science", "COMEDK_Rank": 4049, "KCET_Rank": 1800 },
    { "College": "BMS College of Engineering", "Branch": "Electronics (ECE)", "COMEDK_Rank": 5302, "KCET_Rank": 3500 },

    { "College": "Dayananda Sagar (DSCE)", "Branch": "Computer Science (CSE)", "COMEDK_Rank": 5479, "KCET_Rank": 4500 },
    { "College": "Dayananda Sagar (DSCE)", "Branch": "AI & Machine Learning", "COMEDK_Rank": 6201, "KCET_Rank": 5500 },
    { "College": "Dayananda Sagar (DSCE)", "Branch": "Electronics (ECE)", "COMEDK_Rank": 10951, "KCET_Rank": 9000 },

    { "College": "Bangalore Inst. of Technology", "Branch": "Computer Science (CSE)", "COMEDK_Rank": 9177, "KCET_Rank": 6000 },
    { "College": "Bangalore Inst. of Technology", "Branch": "Electronics (ECE)", "COMEDK_Rank": 19284, "KCET_Rank": 15000 },

    # --- TIER 2: HIGH VALUE OPTIONS ---
    { "College": "CMR Institute of Tech (CMRIT)", "Branch": "Computer Science (CSE)", "COMEDK_Rank": 18084, "KCET_Rank": 14000 },
    { "College": "CMR Institute of Tech (CMRIT)", "Branch": "AI & Machine Learning", "COMEDK_Rank": 22998, "KCET_Rank": 18000 },
    { "College": "CMR Institute of Tech (CMRIT)", "Branch": "Information Science (ISE)", "COMEDK_Rank": 26492, "KCET_Rank": 19000 },
    { "College": "CMR Institute of Tech (CMRIT)", "Branch": "Electronics (ECE)", "COMEDK_Rank": 38765, "KCET_Rank": 32000 },

    { "College": "Sir M Visvesvaraya (Sir MVIT)", "Branch": "Computer Science (CSE)", "COMEDK_Rank": 22026, "KCET_Rank": 15000 },
    { "College": "Sir M Visvesvaraya (Sir MVIT)", "Branch": "Information Science (ISE)", "COMEDK_Rank": 30150, "KCET_Rank": 22000 },
    { "College": "Sir M Visvesvaraya (Sir MVIT)", "Branch": "Electronics (ECE)", "COMEDK_Rank": 34643, "KCET_Rank": 30000 },

    { "College": "RNS Institute (RNSIT)", "Branch": "Computer Science (CSE)", "COMEDK_Rank": 30222, "KCET_Rank": 16000 },
    { "College": "RNS Institute (RNSIT)", "Branch": "Electronics (ECE)", "COMEDK_Rank": 40410, "KCET_Rank": 28000 },

    { "College": "KLE Tech (Hubballi)", "Branch": "Computer Science (CSE)", "COMEDK_Rank": 31388, "KCET_Rank": 13000 },
    { "College": "KLE Tech (Hubballi)", "Branch": "Electronics (ECE)", "COMEDK_Rank": 54028, "KCET_Rank": 25000 },

    # --- TIER 3 & UNIVERSITIES (Safety Options) ---
    { "College": "Dayananda Sagar Academy (DSATM)", "Branch": "Computer Science (CSE)", "COMEDK_Rank": 35187, "KCET_Rank": 20000 },
    { "College": "JSS Academy (JSSATE)", "Branch": "Computer Science (CSE)", "COMEDK_Rank": 36550, "KCET_Rank": 18000 },
    { "College": "Alliance University", "Branch": "Computer Science (CSE)", "COMEDK_Rank": 42800, "KCET_Rank": 40000 },
    { "College": "Presidency University", "Branch": "Computer Science (CSE)", "COMEDK_Rank": 46905, "KCET_Rank": 45000 },
    { "College": "Acharya Institute (AIT)", "Branch": "Computer Science (CSE)", "COMEDK_Rank": 51728, "KCET_Rank": 25000 },
    { "College": "SJB Institute of Tech", "Branch": "Computer Science (CSE)", "COMEDK_Rank": 53748, "KCET_Rank": 28000 },
    { "College": "KLS Gogte (Belgaum)", "Branch": "Computer Science (CSE)", "COMEDK_Rank": 53966, "KCET_Rank": 22000 },
    { "College": "Global Academy (GAT)", "Branch": "Computer Science (CSE)", "COMEDK_Rank": 75438, "KCET_Rank": 45000 },
    { "College": "JNNCE Shivamogga", "Branch": "Computer Science (CSE)", "COMEDK_Rank": 101301, "KCET_Rank": 60000 },
    { "College": "Sahyadri (Mangalore)", "Branch": "Computer Science (CSE)", "COMEDK_Rank": 104962, "KCET_Rank": 70000 }
]

df = pd.DataFrame(master_data)
# Fill missing columns
df['JEE_Rank'] = 999999 # Placeholder
if 'Category' not in df.columns: df['Category'] = 'OPEN'
if 'Gender' not in df.columns: df['Gender'] = 'Gender-Neutral'

print("✅ Database Loaded: All 2025 COMEDK/KCET Cutoffs Ready.")

# --- 3. TRAIN THE AI (Marks -> Rank) ---
# Synthetic data based on 2024/25 Trends
kcet_train = pd.DataFrame({'Marks': [175, 170, 160, 150, 140, 130, 120, 110, 100, 90, 80, 70, 60, 50], 'Rank': [10, 50, 200, 500, 1500, 3500, 6000, 10000, 18000, 30000, 45000, 65000, 90000, 120000]})
comedk_train = pd.DataFrame({'Marks': [170, 160, 150, 140, 130, 120, 110, 100, 90, 80, 70, 60, 50], 'Rank': [15, 100, 400, 1200, 2500, 4500, 7500, 12000, 19000, 28000, 40000, 55000, 70000]})
jee_train = pd.DataFrame({'Marks': [280, 260, 240, 220, 200, 180, 160, 140, 120, 100, 80, 60], 'Rank': [50, 500, 1500, 3500, 6500, 12000, 22000, 38000, 65000, 110000, 180000, 300000]})

kcet_model = make_pipeline(PolynomialFeatures(3), LinearRegression())
kcet_model.fit(kcet_train[['Marks']], kcet_train['Rank'])

comedk_model = make_pipeline(PolynomialFeatures(3), LinearRegression())
comedk_model.fit(comedk_train[['Marks']], comedk_train['Rank'])

jee_model = make_pipeline(PolynomialFeatures(3), LinearRegression())
jee_model.fit(jee_train[['Marks']], jee_train['Rank'])

print("🧠 AI Trained: Rank Prediction Logic Ready.")

# --- 4. PREDICTION ENGINE ---
def predict_my_college(exam, marks):
    input_data = pd.DataFrame({'Marks': [marks]})

    if exam == 'KCET':
        pred_rank = int(kcet_model.predict(input_data)[0])
    elif exam == 'COMEDK':
        pred_rank = int(comedk_model.predict(input_data)[0])
    else:
        pred_rank = int(jee_model.predict(input_data)[0])

    if pred_rank < 1: pred_rank = 1

    col_name = f"{exam}_Rank"
    candidates = df[df[col_name] < 999999].copy()
    eligible = candidates[candidates[col_name] >= pred_rank].sort_values(by=col_name)

    def get_status(cutoff, my_rank):
        if my_rank < (cutoff * 0.8): return "🟢 Safe"
        if my_rank < cutoff: return "🟡 Likely"
        return "🔴 Borderline"

    eligible['Chances'] = eligible[col_name].apply(lambda x: get_status(x, pred_rank))
    return pred_rank, eligible[['College', 'Branch', col_name, 'Chances']].head(15)

# --- 5. USER INTERFACE ---
print("\n🚀 AI COLLEGE PREDICTOR (2025 EDITION)")
print("=========================================")

exam_widget = widgets.Dropdown(options=['KCET', 'COMEDK', 'JEE'], description='Exam:')
marks_widget = widgets.IntText(value=100, description='Marks:')
btn_predict = widgets.Button(description='Predict Results', button_style='primary')
output_area = widgets.Output()

def on_click_predict(b):
    with output_area:
        clear_output()
        rank, results = predict_my_college(exam_widget.value, marks_widget.value)
        print(f"📊 Predicted Rank: ~{rank:,}")
        if results.empty:
            print("⚠️ Rank too low for top colleges in database.")
        else:
            display(results)

btn_predict.on_click(on_click_predict)
display(exam_widget, marks_widget, btn_predict, output_area)

✅ Database Loaded: All 2025 COMEDK/KCET Cutoffs Ready.
🧠 AI Trained: Rank Prediction Logic Ready.

🚀 AI COLLEGE PREDICTOR (2025 EDITION)


Dropdown(description='Exam:', options=('KCET', 'COMEDK', 'JEE'), value='KCET')

IntText(value=100, description='Marks:')

Button(button_style='primary', description='Predict Results', style=ButtonStyle())

Output()